<a href="https://colab.research.google.com/github/StePetro/ComputerVisionProject/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SETUP ENVIRONMENT 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


In [ ]:
!apt-get install -qq xattr
!pip install scikit-learn-extra
!pip install flask-ngrok

In [ ]:
import os
import datetime
import numpy as np
import glob
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as KK
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython.display import display
from keras.regularizers import l2
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import itertools
from PIL import Image, ImageDraw, ImageOps
from IPython.display import display
from time import sleep
import sklearn
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import normalize, minmax_scale
from sklearn_extra.cluster import KMedoids
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,request, abort

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.util.tf_export import keras_export
import xattr

import os
import datetime
from numpy import genfromtxt
from csv import reader
import pandas as pd
import bisect
from numpy import save
import scipy as sc
from tqdm import tqdm
import heapq
import random
import itertools as itool
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

DATA_PATH = '/content/gdrive/My Drive/Data/'
MODELS_PATH = DATA_PATH + 'models'
FACIAL_EXPRESSION_PATH = '/content/dataset'
MIRFLICKR_PATH = "/content/distractor"
FEATURES_PATH = DATA_PATH + "features"
CATEGORICAL_LABELS = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprise", 99:"Distractor"}
FACIAL_EXPRESSION_SETS = ["Training", "PrivateTest", "PublicTest"]
WHITELIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

BATCH_SIZE = 64
BUFFER_SIZE = BATCH_SIZE
VALIDATION_SPLIT = 0.2
SEED = 76


2.4.0


In [ ]:
!unzip '/content/gdrive/My Drive/Data/facial_expression.zip' -d "/content"
!unzip '/content/gdrive/My Drive/Data/distractor.zip' -d "/content"

#DATASET LOADING AND PREPROCESSING

In order to mantain filenames corresponding to the images in the dataset, it is necessary to override the *image_dataset_from_directory* function according to the code at the following link: [image_dataset_from_directory_override](https://github.com/tensorflow/tensorflow/blob/v2.3.0/tensorflow/python/keras/preprocessing/image_dataset.py#L34-L206)

In [ ]:
"""
IMAGE_DATASET_FROM_DIRECTORY FUNCTION OVERRIDE
"""

def image_dataset_from_directory(directory,
                                 labels='inferred',
                                 label_mode='int',
                                 class_names=None,
                                 color_mode='rgb',
                                 batch_size=32,
                                 image_size=(256, 256),
                                 shuffle=True,
                                 seed=None,
                                 validation_split=None,
                                 subset=None,
                                 interpolation='bilinear',
                                 follow_links=False):
  
  if labels != 'inferred':
    if not isinstance(labels, (list, tuple)):
      raise ValueError(
          '`labels` argument should be a list/tuple of integer labels, of '
          'the same size as the number of image files in the target '
          'directory. If you wish to infer the labels from the subdirectory '
          'names in the target directory, pass `labels="inferred"`. '
          'If you wish to get a dataset that only contains images '
          '(no labels), pass `label_mode=None`.')
    if class_names:
      raise ValueError('You can only pass `class_names` if the labels are '
                       'inferred from the subdirectory names in the target '
                       'directory (`labels="inferred"`).')
  if label_mode not in {'int', 'categorical', 'binary', None}:
    raise ValueError(
        '`label_mode` argument must be one of "int", "categorical", "binary", '
        'or None. Received: %s' % (label_mode,))
  if color_mode == 'rgb':
    num_channels = 3
  elif color_mode == 'rgba':
    num_channels = 4
  elif color_mode == 'grayscale':
    num_channels = 1
  else:
    raise ValueError(
        '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
        'Received: %s' % (color_mode,))
  interpolation = image_preprocessing.get_interpolation(interpolation)
  dataset_utils.check_validation_split_arg(
      validation_split, subset, shuffle, seed)

  if seed is None:
    seed = np.random.randint(1e6)
  image_paths, labels, class_names = dataset_utils.index_directory(
      directory,
      labels,
      formats=WHITELIST_FORMATS,
      class_names=class_names,
      shuffle=shuffle,
      seed=seed,
      follow_links=follow_links)

  if label_mode == 'binary' and len(class_names) != 2:
    raise ValueError(
        'When passing `label_mode="binary", there must exactly 2 classes. '
        'Found the following classes: %s' % (class_names,))

  image_paths, labels = dataset_utils.get_training_or_validation_split(
      image_paths, labels, validation_split, subset)

  dataset = paths_and_labels_to_dataset(
      image_paths=image_paths,
      image_size=image_size,
      num_channels=num_channels,
      labels=labels,
      label_mode=label_mode,
      num_classes=len(class_names),
      interpolation=interpolation)
  if shuffle:
    # Shuffle locally at each iteration
    dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names
  return dataset, image_paths

def paths_and_labels_to_dataset(image_paths,
                                image_size,
                                num_channels,
                                labels,
                                label_mode,
                                num_classes,
                                interpolation):
  """Constructs a dataset of images and labels."""
  path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
  img_ds = path_ds.map(
      lambda x: path_to_image(x, image_size, num_channels, interpolation))
  if label_mode:
    label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
    img_ds = dataset_ops.Dataset.zip((img_ds, label_ds))
  return img_ds


def path_to_image(path, image_size, num_channels, interpolation):
  img = io_ops.read_file(path)
  img = image_ops.decode_image(
      img, channels=num_channels, expand_animations=False)
  img = image_ops.resize_images_v2(img, image_size, method=interpolation)
  img.set_shape((image_size[0], image_size[1], num_channels))
  return img

In [ ]:
"""
LOAD FACIAL EXPRESSION DATASET
"""


training_dataset_noshuffle, training_dataset_noshuffle_path = image_dataset_from_directory(directory=FACIAL_EXPRESSION_PATH+"/Training", labels="inferred", label_mode="int", 
                                                          color_mode="rgb", batch_size=BATCH_SIZE, image_size=(160,160), shuffle=False, seed = SEED)
  
public_test_dataset, public_test_dataset_path = image_dataset_from_directory(directory=FACIAL_EXPRESSION_PATH+"/PublicTest", labels="inferred", label_mode="int", color_mode="rgb",
                                                  batch_size=BATCH_SIZE, image_size=(160,160), shuffle=False, seed = SEED)
  
private_test_dataset, private_test_dataset_path = image_dataset_from_directory(directory=FACIAL_EXPRESSION_PATH+"/PrivateTest", labels="inferred", label_mode="int", color_mode="rgb",
                                                  batch_size=BATCH_SIZE, image_size=(160,160), shuffle=False, seed = SEED)

train_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,   
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    FACIAL_EXPRESSION_PATH+"/Training",
    target_size=(160,160),
    batch_size=BATCH_SIZE,
    color_mode="rgb",
    class_mode='sparse',
    shuffle=True,
    seed=SEED,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    FACIAL_EXPRESSION_PATH+"/Training", 
    target_size=(160,160),
    batch_size=BATCH_SIZE,
    color_mode="rgb",
    class_mode='sparse',
    seed=SEED,
    subset='validation')

training_dataset_noshuffle_path =[s.replace('/content','/content/gdrive/MyDrive/Data/facial_expression') for s in training_dataset_noshuffle_path ]
public_test_dataset_path =[s.replace('/content','/content/gdrive/MyDrive/Data/facial_expression') for s in public_test_dataset_path ]
private_test_dataset_path =[s.replace('/content','/content/gdrive/MyDrive/Data/facial_expression') for s in private_test_dataset_path ]

In [ ]:
"""
LOAD MIRFLICKR25 DATASET
"""

mirflickr_dataset, mirflickr_dataset_path = image_dataset_from_directory(directory=MIRFLICKR_PATH, labels="inferred", label_mode="int", color_mode="rgb", batch_size=BATCH_SIZE, image_size=(160,160), shuffle=False, seed=SEED)
mirflickr_dataset_path =[s.replace('/content','/content/gdrive/MyDrive/Data/mirflickr25k') for s in mirflickr_dataset_path ]

Found 25000 files belonging to 1 classes.


In [ ]:
"""
DATASET NORMALIZATION
"""

def preprocess(images, labels):
  # rescales from [0, 255] to [-1, 1], equivalent to:  images = (images / 127.5) - 1
  images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
  return images, labels

# apply prepocessing to datasets (lazy operation)

training_dataset_noshuffle = training_dataset_noshuffle.map(preprocess, deterministic=True)
public_test_dataset = public_test_dataset.map(preprocess, deterministic=True)
private_test_dataset = private_test_dataset.map(preprocess, deterministic=True)
mirflickr_dataset = mirflickr_dataset.map(preprocess, deterministic=True)

In [ ]:
"""
PREFETCH
Prefetch is used in order to speed up operations such as feature extraction.
"""

training_dataset_noshuffle = training_dataset_noshuffle.prefetch(buffer_size=BUFFER_SIZE)
public_test_dataset = public_test_dataset.prefetch(buffer_size=BUFFER_SIZE)
private_test_dataset = private_test_dataset.prefetch(buffer_size=BUFFER_SIZE)
mirflickr_dataset = mirflickr_dataset.prefetch(buffer_size=BUFFER_SIZE)

In [ ]:
"""
WRITE LABELS IN labels.csv FILE
"""

distractorlabels=np.full((25000,1),99)

with open(os.path.join(FEATURES_PATH, "labels.csv"),"ab") as f:

  np.savetxt(f, np.concatenate([y for x, y in training_dataset_noshuffle], axis=0), delimiter=",", fmt="%s")
  np.savetxt(f, np.concatenate([y for x, y in public_test_dataset], axis=0), delimiter=",", fmt="%s")
  #np.savetxt(f, np.concatenate([y for x, y in private_test_dataset], axis=0), delimiter=",", fmt="%s")
  np.savetxt(f, distractorlabels, delimiter=",", fmt="%s")

#STEP 1: PP-INDEX IMPLEMENTATION


##Permutations

###Pivots Choice

In [ ]:
def extract_random_pivots(input_file, pivots_file, N):
  """
  Extract N random pivots from input_file and save them in output_file
  """
  
  #Extract N rows at random without opening the entire file
  with open(os.path.join(FEATURES_PATH,input_file)) as fin:
      sample = heapq.nlargest(N, fin, key=lambda L: random.random())

  # Delete old pivots_file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,pivots_file)):
   os.remove(os.path.join(BASE_PP_INDEX_PATH,pivots_file))

  #Save new pivots_file
  with open(os.path.join(BASE_PP_INDEX_PATH,pivots_file), 'w+') as piv:
    for s in sample:
       piv.write(s)

In [ ]:
def extract_k_medoids_pivots(input_file, pivots_file, aproximation_size, N):
  """
  Extract N pivots from input_file unsing in memory k-medoids algorithm and save them in output_file
  """
  
  #Extract N rows at random without opening the entire file
  with open(os.path.join(FEATURES_PATH,input_file)) as fin:
      strings_sample = heapq.nlargest(aproximation_size, fin, key=lambda L: random.random())

  # Parse to remove the string path from the samples
  sample = np.array([np.float64(s.split(",")[1:]) for s in strings_sample])

  # Take the indexes of the cluster centers
  pivots_index = KMedoids(N,"cosine").fit(sample[:,1:]).medoid_indices_

  # Use the clusters centers as pivots
  pivots = sample[pivots_index]
  pivots = np.column_stack([np.zeros(N).reshape(-1,1), pivots])

  # Delete old pivots_file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,pivots_file)):
    os.remove(os.path.join(BASE_PP_INDEX_PATH,pivots_file))

  #Save new pivots_file
  with open(os.path.join(BASE_PP_INDEX_PATH,pivots_file), 'w+') as piv:
    np.savetxt(piv,pivots,delimiter=",")     

###Permutation Computation

In [ ]:
def create_permutation_dataset(input_file, output_file, pivots_file, chunksize):
  """
  Compute pivots permutation for each element and save it in a file
  """

  #Take pivots
  df_pivots=pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,pivots_file), sep=',',header=None )
  pivots = df_pivots.to_numpy()

  #Take features
  df_features=pd.read_csv(os.path.join(FEATURES_PATH,input_file), sep=',',header=None, chunksize = chunksize)

  # Delete old permutation file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,output_file)):
    os.remove(os.path.join(BASE_PP_INDEX_PATH,output_file))

  #Compute permutations
  for chunk in tqdm(df_features):
    chunk = np.array(chunk)

    distances_list_chunk = []
    for elem in chunk: 

      distances_list = []

      #Compute distance list between elem and pivots
      for piv in pivots:

        dist = cosine_distances(np.array(elem[1:]).reshape(1,-1),np.array(piv[1:]).reshape(1,-1))[0]
        distances_list.extend(dist)
        distances_from_pivots = np.array(distances_list)

      # Reorder the distance list (to obtain the permutation)
      distances_list_chunk.append(np.array(pd.DataFrame(np.argsort(distances_from_pivots)).T).reshape(-1))

    # Save permutations to csv file 
    pd.DataFrame(distances_list_chunk).to_csv(os.path.join(BASE_PP_INDEX_PATH,output_file), mode='a', index = False, header=False)

In [ ]:
def compute_permutation_prefix(query, dim_prefix, pivots):
  """
  Compute prefix of a given query 
  """

  permutations_list=[]
  distances_list = []

  # Compute distance list between query and pivots 
  for piv in pivots:
    dist = cosine_distances(np.array(query[1:]).reshape(1,-1),np.array(piv[1:]).reshape(1,-1))[0]
    distances_list.extend(dist)
  
  # Compute permutation: reoder the distance list 
  distances_from_pivots = np.array(distances_list)
  permutations_list.extend(np.argsort(distances_from_pivots))
  # Return the prefix (=the first dim_prefix elements of permutation)
  return np.array(permutations_list)[:dim_prefix]

##Prefix Tree

In [ ]:
class PrefixTree(object):
    """
    Prefix tree implementation
    """
    
    def __init__(self, value: str, is_leaf = False, is_last_pivot = False):
        # Value of the node
        self.value = value
        self.children = []
        # If this is a leaf node
        self.is_leaf = is_leaf
        self.is_last_pivot = is_last_pivot
        # How many images have this prefix
        self.counter = 1
    

    def add(root, permutation, position_index):
        """
        Adding a permutation in the prefix tree structure
        """
        node = root
        for idx, pivot in enumerate(permutation):
            found_in_child = False
            # Search for the pivot in the children of the present `node`
            for child in node.children:
                if child.value == pivot:
                    # Increase the counter by 1 to keep track that another
                    # permutation has hit
                    child.counter += 1
                    # Point node to the child that contains this pivot
                    node = child
                    found_in_child = True                    
            # If pivot is not find add a new chlid
            if not found_in_child:
                new_node = PrefixTree(pivot)
                node.children.append(new_node)
                # Point node to the new child
                node = new_node
            if idx == (len(permutation) - 1):
                #last
                node.children.append(PrefixTree(position_index, is_leaf = True))
            # Last prefix's pivot
        node.is_last_pivot = True


    def find_prefix(root, prefix: str, k):
      """
      Find the prefix subtree with that prefix and at least k leaves
      """
      node = root
      # If the root node has no children, then return False.
      if not root.children:
        return False

      for pivot in prefix:
        pivot_not_found = True

        # Search through all the children
        for child in node.children:
                  
          if child.value == pivot:
            pivot_not_found = False        
            if(child.counter < k):
              # If child leaves < k return parent to have at least k leaves
              return node
            node = child
            break

        # Return False anyway when not find a pivot
        if pivot_not_found:
          return False

      return node

    def get_leaves(node):
      """
      Get prefix tree's leaves in order
      """
      leaves = []
      # Check each children of a node
      for child in node.children:
        if child.is_leaf:
          # Take the leaf value
          leaves.append(child.value)
        else:
          leaves.extend(PrefixTree.get_leaves(child))
      return leaves

    
    def extract_parent_pointers(node):
      """
      Extract leaves parents
      """
      parent_pointers = []
      # Check each children of a node
      for child in node.children:
        if child.is_last_pivot:  
          # If is_last_pivot = True it means that the node children are leaves
          parent_pointers.append(child) 
        if not child.is_last_pivot:
          parent_pointers.extend(PrefixTree.extract_parent_pointers(child))
      return parent_pointers


    def merge_leaves(node):
      """
      Merge pointers to reflect new order
      """
      parent_pointers = PrefixTree.extract_parent_pointers(node)
      #Use idx to save how many elements merged in the leaves 
      idx = 0
      # For each leaf parent
      for parent in parent_pointers:
        # If there are more than one leaf
        if len(parent.children)!= 1:
          # Merge and save start-idx and end-idx
          size = len(parent.children)
          parent.children = [ PrefixTree(idx, is_leaf = True),  
                              PrefixTree(idx + len(parent.children) - 1, is_leaf = True) ]
          idx += size
        else:
          # Only one element in leaf
          parent.children = [ PrefixTree(idx, is_leaf = True)]  
          idx += 1

 
    def print_tree(node, _prefix="", _last=True):
      """
      Print prefix tree's structure
      """
      print(_prefix, "`- " if _last else "|- ", node.value, sep="")
      _prefix += "   " if _last else "|  "
      child_count = len(node.children)
      for i, child in enumerate(node.children):
        _last = i == (child_count - 1)
        PrefixTree.print_tree(child, _prefix, _last)

  
    def add_all(root, permutations, prefix_len, base_index = 0):
      '''
      Insert in the tree all the permutations cutting them according to prefix_len
      '''
      for index, prefix in enumerate(permutations):
        PrefixTree.add(root, np.array(prefix).reshape(-1)[:prefix_len], base_index + index)


    def add_from_csv(root, permutation_file, prefix_len, chunksize):
      """
      Insert in the tree all the permutations cutting them according to prefix_len
      from csv permutation_file in chunks of chunksize
      """
      permutations_chunks = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,permutation_file), sep=',', header=None, chunksize = chunksize)
      for i, permutations_chunk in tqdm(enumerate(permutations_chunks)):
        PrefixTree.add_all(root, np.array(permutations_chunk), prefix_len, chunksize*i)



##Reordering

In [ ]:
def reorder_datastore_in_memory(input_file, output_file, indexing_mapping):
  """
  Reorder the rows of a csv file (in memory) according to indexing_mapping 
  """
  # Read file
  features = pd.read_csv(os.path.join(FEATURES_PATH,input_file), sep=',', header=None)
  # Insert a new column containing the Indexing
  features.insert(0, 'Indexing', indexing_mapping)
  # Sort the rows by the Indexing 
  features = features.sort_values(by=['Indexing']).drop(labels = 'Indexing', axis=1)  
  # Save reordered rows in new file
  features.to_csv(os.path.join(BASE_PP_INDEX_PATH,output_file), mode='w', index = False, header=False)

In [ ]:
  """
  Reorder the rows of a csv file (on disk) according to leaves_list
  """
  # Delete old file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,output_file)):
    os.remove(os.path.join(BASE_PP_INDEX_PATH,output_file))

  # For each leaf in the list
  for leaf in tqdm(leaves_list):

    # Read a row and save it to output_file (append mode)
    df_features = pd.read_csv(os.path.join(FEATURES_PATH,input_file), sep=',', header=None, skiprows= leaf, nrows=1).to_csv(os.path.join(BASE_PP_INDEX_PATH,output_file), mode='a', index = False, header=False)

##PP-Index Creation

In [ ]:
def create_PP_index(n_pivots, prefix_len, features_file, pivots_file, permutations_file, reordered_features_file, labels_file, reordered_labels_file, k_medoids = False, aproximation_size = 25000, chunksize = 1000, on_disk = False, num_indexes = 1):
  '''
  Creates the PP Index and returns the prefix tree
  Pass only file name without extention
  '''

  prefix_trees = []

  for idx in range(num_indexes):
    # Pivot extraction from features space
    if k_medoids:
      extract_k_medoids_pivots(features_file + ".csv",pivots_file + "_" + str(idx) + ".csv", aproximation_size, n_pivots)
    else:
      extract_random_pivots(features_file + ".csv", pivots_file + "_" + str(idx) + ".csv", n_pivots)

    # Dataset of images permutations of indexes 
    create_permutation_dataset(features_file + ".csv", permutations_file + "_" + str(idx) + ".csv", pivots_file + "_" + str(idx) + ".csv", chunksize) 

    # Creates Empty Prefix Tree
    prefix_tree_root = PrefixTree("*")
    # Adds all images pivots permutations in "permutations.csv"
    PrefixTree.add_from_csv(prefix_tree_root, permutations_file + "_" + str(idx) + ".csv", prefix_len, chunksize = chunksize)

    # Get images indexes ordered by the prefix tree
    indexes_reordered_by_prefix_tree = PrefixTree.get_leaves(prefix_tree_root)
    print(indexes_reordered_by_prefix_tree)

    if on_disk:
      # On Disk features and labels reordering (Slow using colab and google drive)
      reorder_datastore_on_disk(features_file + ".csv", reordered_features_file + "_" + str(idx) + ".csv", indexes_reordered_by_prefix_tree)
      reorder_datastore_on_disk(labels_file + ".csv", reordered_labels_file + "_" + str(idx) + ".csv", indexes_reordered_by_prefix_tree)
    else:
      # Reorder indexes to obtain a mapping necessary to quick memory sort
      indexing_mapping = [indexes_reordered_by_prefix_tree.index(i) for i in tqdm(range(len(indexes_reordered_by_prefix_tree)))]
      print(indexing_mapping)

      # In memory features and labels reordering
      reorder_datastore_in_memory(features_file + ".csv", reordered_features_file + "_" + str(idx) + ".csv", indexing_mapping)
      reorder_datastore_in_memory(labels_file + ".csv", reordered_labels_file + "_" + str(idx) + ".csv", indexing_mapping)

    # Merge leaves in order to have only start and end index for a certain permutation
    PrefixTree.merge_leaves(prefix_tree_root)
    #PrefixTree.print_tree(prefix_tree_root)
    prefix_trees.append(prefix_tree_root)

  return prefix_trees

##Search

In [ ]:
def extract_features(feature_extractor, images):
  """
  Extract features from a given image
  """
  # Extract features
  features = feature_extractor.predict(images, batch_size=BATCH_SIZE)
  # Rescale between 0 and 1
  features = minmax_scale(features,axis=1)
  # Add a dummy element in position 0 (for compatibility with other functions)
  features = np.column_stack([[[0]], features])

  return features

In [ ]:
def search_candidates(query_features, prefix_tree, pivots, Z, features_file, labels_file, precomputed_query_prefix = None):
  """
  Explore the prefix tree to find candidates for a query
  """
  # If the query prefix is not given, compute 
  if precomputed_query_prefix is None:
    query_prefix = compute_permutation_prefix(query_features, PREFIX_LEN, pivots)
  else:
    query_prefix = precomputed_query_prefix
  
  # Find the smallest subtree containing at least Z elements in the leaves
  subtree = PrefixTree.find_prefix(prefix_tree, query_prefix, Z)
  # PrefixTree.print_tree(subtree)
  leaves = PrefixTree.get_leaves(subtree)

  # Read features and the corrispondet labels according to the subtree leaves
  df_features = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,features_file), sep=',', header=None, skiprows= leaves[0], nrows=(leaves[-1] - leaves[0] +1)).to_numpy()
  df_labels = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,labels_file), sep=',', header=None, skiprows= leaves[0], nrows=(leaves[-1] - leaves[0] +1)).to_numpy().reshape(-1)

  return df_features, df_labels

In [ ]:
def k_nn_search(query_features, candidates, candidates_labels, k, from_zip = False, attachID = False):
  '''
  Search for k-nearest images to the query by features cosine_similarity
  Returns paths, similarities, labels
  '''
  similarity_list =[]


  # Computes cosine similarity for every candidate
  print(f"\nCandidates: {len(candidates)}")
  for candidate in candidates:
    dist = cosine_similarity(np.array(query_features[1:]).reshape(1,-1),np.array(candidate[1:]).reshape(1,-1))[0]
    similarity_list.extend(dist)

  # Reorder the similarity list 
  similarities_from_pivots = np.array(similarity_list)
  sorted_list = np.argsort(similarities_from_pivots)[::-1] #reverse

  paths = []
  BASE = "https://drive.google.com/uc?export=view&id="

  for idx in sorted_list[:k]:

    c_path = candidates[idx,0]

    # If we wanto to read from drive
    if attachID:
      # Extract images paths
      id=xattr.getxattr(c_path, "user.drive.id")
      c_path = BASE + str(id).split("'")[1]
    
    # If we wanto to read from zip
    if from_zip:
      c_path.replace('/content/gdrive/MyDrive/Data/mirflickr25k', '/content')
      c_path.replace('/content/gdrive/MyDrive/Data/facial_expression', '/content')
      paths.append(c_path)
    else:
      paths.append(c_path)

  return paths, np.array(similarity_list)[sorted_list[:k]], np.array(candidates_labels)[sorted_list[:k]]

In [ ]:
def search(queries_features, K, pivot_file, reordered_features_file, reordered_labels_file, from_zip = False, attachID = False, query_perturbation = False, perturbation_len = 3):
  '''
  Search for k best images similar to queries using PP-Index
  Returns topk_images_list, topk_scores_list, topk_labels_list
  '''

  topk_images_list = []
  topk_scores_list = []
  topk_labels_list = []

  for query in tqdm(queries_features): 
    
    topk_to_be_merged = np.array([["dummy_path", "-1", "dummy_label"]])

    # Merge all results from every PP Index if multiple are used
    for idx, prefix_tree in enumerate(PREFIX_TREES):

      # File names handling
      pivots = np.array(pd.read_csv(os.path.join(BASE_PP_INDEX_PATH, pivot_file + "_" + str(idx) + ".csv"), sep=',',header=None))
      reordered_features = reordered_features_file + "_" + str(idx) + ".csv"
      reordered_labels = reordered_labels_file + "_" + str(idx) + ".csv"

      # If query perturbation is used
      if query_perturbation:

        # Compute all permutations of the first perturbation_len values of the query prefix
        query_prefix = compute_permutation_prefix(query, PREFIX_LEN, pivots)
        to_be_perturbated = query_prefix[:perturbation_len]
        permutations = list(itool.permutations(to_be_perturbated))

        # Search candidates for the first permutation. Save the candidates together with their labels
        query_prefix[:perturbation_len] = np.array(permutations[0])
        new_candidates, candidates_labels = search_candidates(query, prefix_tree, pivots, N_CANDIDATES, reordered_features, reordered_labels, query_prefix)
        candidates = np.column_stack([np.array(candidates_labels).reshape(-1,1), np.array(new_candidates)])

        # Search candidates for all the other permutations. Save the candidates together with their labels
        for permutation in permutations[1:]:

          query_prefix[:perturbation_len] = np.array(permutation)
          new_candidates, candidates_labels = search_candidates(query, prefix_tree, pivots, N_CANDIDATES, reordered_features, reordered_labels, query_prefix)
          # Merge candidates for all the permutations
          candidates = np.concatenate((np.column_stack([np.array(candidates_labels).reshape(-1,1), np.array(new_candidates)]), candidates))

        # Remove dupliacates and separate candidates labels from candidates
        unique_candidates = np.unique(candidates.astype("str"), axis=0)
        candidates_labels = unique_candidates[:,0].astype('int') 
        candidates = unique_candidates[:,1:]

      # Query perturbation is not used
      else:
        candidates, candidates_labels = search_candidates(query, prefix_tree, pivots, N_CANDIDATES, reordered_features, reordered_labels)

      # k_nn_search to obtain topk_paths, topk_scores, topk_lables
      topk_paths,  topk_scores, topk_labels= k_nn_search(query, candidates, candidates_labels, K, from_zip, attachID)

      # Concatenate top k results obtained from each index
      topk_to_be_merged = np.concatenate((np.array(list(zip(topk_paths,topk_scores,topk_labels))), topk_to_be_merged))
    
    # Remove duplicates and sort by score
    unique_topk = np.unique(topk_to_be_merged.astype("str"), axis=0)
    unique_topk = unique_topk[unique_topk[:,1].argsort()][::-1]

    # Take top k results
    topk_paths = unique_topk[:K,0]
    topk_scores = unique_topk[:K,1].astype('float64') 
    topk_labels = unique_topk[:K,2].astype('int')

    # If we read from drive
    if not attachID:
      topk_images = [img_to_array(load_img(path, target_size=(PREVIEW_SIZE,PREVIEW_SIZE))) for path in  topk_paths]
      topk_images_list.append(topk_images)
    else:
      topk_images_list.append(topk_paths)    

    topk_scores_list.append(topk_scores)
    topk_labels_list.append(topk_labels)

  return topk_images_list, topk_scores_list, topk_labels_list

##Evaluation

In [ ]:
# utility to resize, pad, and write score on images
def process_images(image, score, is_relevant):
  image = img_to_array(image)

  # to uint8
  # image = ((image + 1) * 127.5).astype(np.uint8)
  image = image.astype(np.uint8)

  # add a red/green flag
  color = (0, 255, 0) if is_relevant else (255, 0, 0)
  flag = np.full((10, PREVIEW_SIZE, 3), fill_value=color, dtype=image.dtype)
  image = np.concatenate((image, flag), axis=0)

  # resize
  image = Image.fromarray(image).convert('RGBA')
  image.thumbnail((PREVIEW_SIZE, PREVIEW_SIZE))  # use PIL to resize the image

  # draw score
  draw = ImageDraw.Draw(image)
  draw.text((3, PREVIEW_SIZE - 12), f'{score:.2f}', anchor='lt', fill=(255, 255, 255, 255))

  # pad the image with transparency
  image = ImageOps.expand(image, 5, fill=(0, 0, 0, 0))
  image = np.array(image)
  return image

# use np.vectorize to apply custom functions to numpy arrays
np_process_image = np.vectorize(process_images, signature='(h,w,c),(),()->(h1,w1,c1)')

# utility function to draw knn results for multiple queries
def show_results(images, scores, is_relevant):
  # images has shape (n_queries, k, H, W, C)
  images = np_process_image(images, scores, is_relevant)
  images = np.concatenate(images, axis=1)  # concatenate queries vertically
  images = np.concatenate(images, axis=1)  # concatenate results horizontally
  display(Image.fromarray(images))

In [ ]:
def evaluate_knn_search(queries_features, queries_labels, k, pivot_file, reordered_features_file, reordered_labels_file, from_zip=False, query_perturbation=False):
  '''
  Evaluate APs and mean AP of all queries
  '''

  # Search 
  topk_images, topk_scores, topk_labels = search(queries_features, k, pivot_file, reordered_features_file, reordered_labels_file, from_zip, query_perturbation = query_perturbation)
  # If a certain retreived image is relevant 
  topk_is_relevant = np.array(topk_labels) == queries_labels.reshape(-1,1)

  # Average Precision Score computation
  aps = np.array([sklearn.metrics.average_precision_score(l, s) for l,s in zip(topk_is_relevant, topk_scores)])
  aps[np.isnan(aps)] = 0
  print('APs per Query:', aps)
  print('mAP:', np.mean(aps))

  # Shows a result grid
  show_results(topk_images, topk_scores, topk_is_relevant)

#STEP 2: FEATURE EXTRACTION FROM FACIAL EXPRESSION DATASET AND MIRFLICKR25 USING FACENET
In this step, features are extracted from both *Facial Expression Dataset* and *MirFlickr Dataset* using the pretrained network *Facenet*. Features are normalized using the *minmax_scale* ([0,1] interval). Files paths are written together with the extracted features in the *pretrained_features_normal.csv* file. Features from *private_test_set* are stored in a different file (*pretrained_features_private_normal.csv*).

##Feature Extraction

In [ ]:
"""
FACENET MODEL LOADING
Facenet model is loaded from the file facenet_keras.h5.
"""

MODEL_PATH='/content/gdrive/My Drive/Data/Facenet/model'
WEIGHTS_PATH= '/content/gdrive/My Drive/Data/Facenet/weights'
facenet = load_model(os.path.join(MODEL_PATH,'facenet_keras.h5'))
facenet.load_weights(os.path.join(WEIGHTS_PATH,'facenet_keras_weights.h5'))
facenet.trainable=False

In [ ]:
"""
FEATURES EXTRACTION FROM TRAINING SET
Features are extracted and normalized using the minmax_scale function (they are
translated into a value in [0,1] interval). File path is added as first column, 
so for each image the features file contains 129 values: path and 128 features 
(in the interval [0,1]).
"""

training_dataset_features = facenet.predict(training_dataset_noshuffle)
training_dataset_features = minmax_scale(training_dataset_features,axis=1)
training_dataset_noshuffle_path = np.array(training_dataset_noshuffle_path)
training_dataset_features = np.column_stack([training_dataset_noshuffle_path,training_dataset_features])

In [ ]:
"""
FEATURES EXTRACTION FROM PUBLIC AND TEST SET
Features are extracted and normalized using the minmax_scale function (they are
translated into a value in [0,1] interval). File path is added as first column, 
so for each image the features file contains 129 values: path and 128 features 
(in the interval [0,1]).
"""

public_test_dataset_features = facenet.predict(public_test_dataset)
public_test_dataset_features = minmax_scale(public_test_dataset_features,axis=1)
private_test_dataset_features = facenet.predict(private_test_dataset)
private_test_dataset_features = minmax_scale(private_test_dataset_features,axis=1)

public_test_dataset_path = np.array(public_test_dataset_path)
private_test_dataset_path = np.array(private_test_dataset_path)

public_test_dataset_features = np.column_stack([public_test_dataset_path,public_test_dataset_features])
private_test_dataset_features = np.column_stack([private_test_dataset_path,private_test_dataset_features])

In [ ]:
"""
FEATURES EXTRACTION FROM MIRFLICKR DATASET
Features are extracted and normalized using the minmax_scale function (they are
translated into a value in [0,1] interval). File path is added as first column, 
so for each image the features file contains 129 values: path and 128 features 
(in the interval [0,1]).
"""

mirflickr_dataset_features = facenet.predict(mirflickr_dataset)
mirflickr_dataset_features = minmax_scale(mirflickr_dataset_features,axis=1)
mirflickr_dataset_path = np.array(mirflickr_dataset_path)
mirflickr_dataset_features = np.column_stack([mirflickr_dataset_path,mirflickr_dataset_features])

##Features Saving

In [ ]:
"""
WRITE TRAINING SET FEATURES IN CSV FILE
Training set features are written into the file pretrained_features_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "pretrained_features_normal.csv"),"ab") as f:

  np.savetxt(f, training_dataset_features, delimiter=",", fmt="%s")

"""
WRITE PUBLIC TEST SET FEATURES IN CSV FILE
Public test set features are written into the file pretrained_features_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "pretrained_features_normal.csv"),"ab") as f:

  np.savetxt(f, public_test_dataset_features, delimiter=",", fmt="%s")


"""
WRITE PRIVATE TEST SET FEATURES IN CSV FILE
Private test set features are written into the file pretrained_features_private_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "pretrained_features_private_normal.csv"),"ab") as f:

  np.savetxt(f, private_test_dataset_features, delimiter=",", fmt="%s")

"""
WRITE MIRFLICKR SET FEATURES IN CSV FILE
Mirflickr set features are written into the file pretrained_features_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "pretrained_features_normal.csv"),"ab") as f:

  np.savetxt(f, mirflickr_dataset_features, delimiter=",", fmt="%s")

#STEP 3: INDEX THE EXTRACTED FEATURES USING PP-INDEX

In [ ]:
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
N_PIVOTS = 10
PREFIX_LEN = 4
CHUNKSIZE = 1000

In [ ]:
# Pivot extraction from features space
pretrained_tree = create_PP_index(N_PIVOTS, PREFIX_LEN, "pretrained_features", "pretrained_pivots", "pretrained_permutations", "pretrained_reordered_features", "labels", "pretrained_reordered_labels", k_medoids = False, chunksize = 1000, on_disk = False, num_indexes = 1)

#STEP 4: MEASURE THE RETRIEVAL PERFORMANCE OF THE NEW IMAGE SEARCH ENGINE

In [ ]:
N_QUERIES = 7
K = 100
N_CANDIDATES = 300
PREVIEW_SIZE = 100

PREFIX_TREES = pretrained_tree
PIVOTS = "pretrained_pivots"
REORDERED_FEATURES = "pretrained_reordered_features"
REORDERED_LABELS = "pretrained_reordered_labels"

test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

# One for each class
query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]

# Random
# query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random

queries_labels = test_Labels[query_indexes]
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "pretrained_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

for fetures, label in zip(queries_features, queries_labels):
  print(CATEGORICAL_LABELS[label])
  display(load_img(fetures[0], target_size=(PREVIEW_SIZE,PREVIEW_SIZE)))

In [ ]:
evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

#STEP 5: FINE-TUNING FACENET FOR FACIAL EXPRESSION DATASET


In [ ]:
"""
FACENET MODEL LOADING
Facenet model is loaded. The base is frozen so that the weights are not modified 
during the first training phase.
"""

MODEL_PATH='/content/gdrive/My Drive/Data/Facenet/model'
WEIGHTS_PATH= '/content/gdrive/My Drive/Data/Facenet/weights'
facenet = load_model(os.path.join(MODEL_PATH,'facenet_keras.h5'))
facenet.load_weights(os.path.join(WEIGHTS_PATH,'facenet_keras_weights.h5'))
facenet.trainable=False

In [ ]:
print(facenet.inputs)
print(facenet.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [ ]:
"""
MODEL CREATION
A fully connected layer is added on the top of Facenet in order to perform the
classification. Softmax is used as activation function. The number of units is 
equal to the number of classes.
"""

model=Sequential()
model.add(facenet)
model.add(Dense(7,activation='softmax'))

In [ ]:
"""
MODEL COMPILATION
For the first training phase, sparse_categorical_crossentropy is used as loss function
and Adam (with LR=1e-3) as optimizer. 
"""

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-3),
              metrics=['acc'])

##Training the classifier on the top of Facenet

Since class distribution is unbalanced, different weights are assigned to different classes according to the formula used in [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights):

\begin{equation}
weight_K= \frac{T}{C*K}
\end{equation}

where: <br>
$T$ = total number of elements <br>
$C$ = total number of classes <br>
$K$ = number of elements of class K <br>
$W_k$ = weight assigned to class k <br><br>
The model is trained for 10 epochs.

In [ ]:
"""
ASSIGN WEIGHTS FOR EACH CLASS
Weights are assigned to the different classes according to the formula defined
above.
"""

train_labels=np.concatenate([y for x, y in training_dataset_noshuffle],axis=0)
weight_for_0=1/(np.count_nonzero(train_labels==0))*train_labels.size/7.0
weight_for_1=1/(np.count_nonzero(train_labels==1))*train_labels.size/7.0
weight_for_2=1/(np.count_nonzero(train_labels==2))*train_labels.size/7.0
weight_for_3=1/(np.count_nonzero(train_labels==3))*train_labels.size/7.0
weight_for_4=1/(np.count_nonzero(train_labels==4))*train_labels.size/7.0
weight_for_5=1/(np.count_nonzero(train_labels==5))*train_labels.size/7.0
weight_for_6=1/(np.count_nonzero(train_labels==6))*train_labels.size/7.0
class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3, 4: weight_for_4, 5: weight_for_5, 6: weight_for_6}

In [ ]:
print(class_weight)

{0: 1.0266046844269623, 1: 9.406618610747051, 2: 1.0010460615781582, 3: 0.5684387684387684, 4: 0.8260394187886636, 5: 0.8491274770777876, 6: 1.293372978330405}


In [ ]:
"""
MODEL FITTING
Model is trained for a limited number of epochs (10). Batch size is equal to 64.
"""

history = model.fit(
      train_generator,
      batch_size = BATCH_SIZE,
      epochs=10,
      validation_data = validation_generator,
      class_weight=class_weight
      )

##Evaluate the performances of classifier

In [ ]:
"""
PLOT TRAINING AND VALIDATION LOSS AND ACCURACY  
"""

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'g', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
"""
GETTING ACCURACY AND OTHER METRICS ON PUBLIC TEST DATASET
In order to obtain the metrics, labels predicted from the model are compared with
the original ones (from public_test_dataset). 
"""

test_labels=np.concatenate([y for x, y in public_test_dataset], axis=0)
test_predictions = model.predict(public_test_dataset)
pred_labels=np.argmax(test_predictions, axis=-1)
print(sklearn.metrics.classification_report(test_labels, pred_labels))

In [ ]:
"""
SAVE BASE MODEL
The model is saved into the file base_classifier02.h5.
"""

model.save(os.path.join(MODELS_PATH,"base_classifier02.h5"))

##Fine Tuning

In [ ]:
"""
UNFREEZING ALL BASE LAYERS
For the second training step, the whole Facenet base is unfrozen, so that all weights
are modified in this phase.
"""

facenet.trainable = True
model=Sequential()
model.add(facenet)
model.add(Dense(7,activation='softmax'))

In [ ]:
"""
MODEL COMPILATION
For this phase, sparse_categorical_crossentropy is used as loss function, while 
the optimizer is RMSprop (with LR=1e-5) because of its higher speed.
"""

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
"""
MODEL FITTING
Batch size is still 64. The maximum number of epochs is 100, however a Callback with 
patience 10 is used to prevent overfitting.
"""

history = model.fit(
      train_generator,
      batch_size = BATCH_SIZE,
      epochs=100,
      validation_data = validation_generator,
      class_weight=class_weight,
      callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience= 10)]
      )

##Evaluate Fine Tuned Network

In [ ]:
"""
PLOT TRAINING AND VALIDATION LOSS AND ACCURACY 
"""

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'g', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
"""
GETTING ACCURACY AND OTHER METRICS ON PUBLIC TEST DATASET
In order to obtain the metrics, labels predicted from the model are compared with
the original ones (from public_test_dataset).
"""

test_labels=np.concatenate([y for x, y in public_test_dataset], axis=0)
test_predictions = model.predict(public_test_dataset)
pred_labels=np.argmax(test_predictions, axis=-1)
print(sklearn.metrics.classification_report(test_labels, pred_labels))

In [ ]:
"""
SAVE THE MODEL
"""

model.save(os.path.join(MODELS_PATH,"fine_tuned_model002.h5"))

In [ ]:
facenet.save(os.path.join(MODELS_PATH,"facenet_ft2.h5"))

#STEP 6: USE THE FINE-TUNED FACENET TO EXTRACT FEATURES FROM FACIAL EXPRESSION DATASET AND MIRFLICKR
In this step, features are extracted from both *Facial Expression Dataset* and *MirFlickr Dataset* using the fine-tuned network. Features are normalized using the *minmax_scale* ([0,1] interval). Files paths are written together with the extracted features in the *finetuned_features_normal.csv* file. Features from *private_test_set* are stored in a different file (*finetuned_features_private_normal.csv*).

##Feature Extraction

In [ ]:
"""
TRAINING SET FEATURE EXTRACTION
Features are extracted and normalized using the minmax_scale function (they are
translated into a value in [0,1] interval). File path is added as first column, 
so for each image the features file contains 129 values: path and 128 features 
(in the interval [0,1]).
"""

training_dataset_features = facenet.predict(training_dataset_noshuffle)
training_dataset_features = minmax_scale(training_dataset_features,axis=1)
training_dataset_noshuffle_path = np.array(training_dataset_noshuffle_path)
training_dataset_features = np.column_stack([training_dataset_noshuffle_path,training_dataset_features])

In [ ]:
"""
PUBLIC AND PRIVATE TEST SET FEATURE EXTRACTION
Features are extracted and normalized using the minmax_scale function (they are
translated into a value in [0,1] interval). File path is added as first column, 
so for each image the features file contains 129 values: path and 128 features 
(in the interval [0,1]).
"""

public_test_dataset_features = facenet.predict(public_test_dataset)
private_test_dataset_features = facenet.predict(private_test_dataset)

public_test_dataset_features = minmax_scale(public_test_dataset_features,axis=1)
public_test_dataset_path = np.array(public_test_dataset_path)
private_test_dataset_features = minmax_scale(private_test_dataset_features,axis=1)
private_test_dataset_path = np.array(private_test_dataset_path)

public_test_dataset_features = np.column_stack([public_test_dataset_path,public_test_dataset_features])
private_test_dataset_features = np.column_stack([private_test_dataset_path,private_test_dataset_features])

In [ ]:
"""
MIRFLICKR DATASET FEATURE EXTRACTION
Features are extracted and normalized using the minmax_scale function (they are
translated into a value in [0,1] interval). File path is added as first column, 
so for each image the features file contains 129 values: path and 128 features 
(in the interval [0,1]).
"""

mirflickr_dataset_features = facenet.predict(mirflickr_dataset)
mirflickr_dataset_features = minmax_scale(mirflickr_dataset_features,axis=1)
mirflickr_dataset_path = np.array(mirflickr_dataset_path)
mirflickr_dataset_features = np.column_stack([mirflickr_dataset_path,mirflickr_dataset_features])

##Features Saving

In [ ]:
"""
WRITE TRAINING SET FEATURES IN CSV FILE
Training set features are written into the file finetuned_features_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "finetuned_features_normal.csv"),"ab") as f:

  np.savetxt(f, training_dataset_features, delimiter=",", fmt="%s")

"""
WRITE PUBLIC TEST SET FEATURES IN CSV FILE
Public test set features are written into the file finetuned_features_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "finetuned_features_normal.csv"),"ab") as f:

  np.savetxt(f, public_test_dataset_features, delimiter=",", fmt="%s")

"""

WRITE PRIVATE TEST SET FEATURES IN CSV FILE
Private test set features are written into the file finetuned_features_private_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "finetuned_features_private_normal.csv"),"ab") as f:

  np.savetxt(f, private_test_dataset_features, delimiter=",", fmt="%s")


"""
WRITE MIRFLICKR SET FEATURES IN CSV FILE
Mirflickr set features are written into the file finetuned_features_normal.csv
"""

with open(os.path.join(FEATURES_PATH, "finetuned_features_normal.csv"),"ab") as f:

  np.savetxt(f, mirflickr_dataset_features, delimiter=",", fmt="%s")

#STEP 7: INDEX THE NEW EXTRACTED FEATURES USING PP-INDEX

In [ ]:
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
N_PIVOTS = 7
PREFIX_LEN = 4
CHUNKSIZE = 1000

In [ ]:
# Pivot extraction from features space
finetuned_tree = create_PP_index(N_PIVOTS, PREFIX_LEN, "finetuned_features", "finetuned_pivots", "finetuned_permutations", "finetuned_reordered_features", "labels", "finetuned_reordered_labels", k_medoids = True, chunksize = 1000, on_disk = False, num_indexes = 1)

#STEP 8: MEASURE THE RETRIEVAL PERFORMANCE OF THE NEW IMAGE SEARCH ENGINE

Qua c'era un errore perchè in realtà prendevate alcune cose dell'indexing pretrained (ad esempio pivots, features del private dataset)

In [ ]:
N_QUERIES = 7
K = 100
N_CANDIDATES = 300
PREVIEW_SIZE = 100

PREFIX_TREES = finetuned_tree
PIVOTS = "finetuned_pivots"
REORDERED_FEATURES = "finetuned_reordered_features"
REORDERED_LABELS = "finetuned_reordered_labels"

test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

# One for each class
query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]

# Random
# query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random

queries_labels = test_Labels[query_indexes]
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "finetuned_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

for fetures, label in zip(queries_features, queries_labels):
  print(CATEGORICAL_LABELS[label])
  display(load_img(fetures[0], target_size=(PREVIEW_SIZE,PREVIEW_SIZE)))

In [ ]:
evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

#STEP 9: COMPARE THE PERFORMANCE OF THE TWO SEARCH ENGINES

In [ ]:
N_QUERIES = 7
K = 100
N_CANDIDATES = 300
PREVIEW_SIZE = 100

test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

# One for each class
query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]

# Random
# query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random

queries_labels = test_Labels[query_indexes]

In [ ]:
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
N_PIVOTS = 7
PREFIX_LEN = 4

In [ ]:
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "pretrained_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

for fetures, label in zip(queries_features, queries_labels):
  print(CATEGORICAL_LABELS[label])
  display(load_img(fetures[0], target_size=(PREVIEW_SIZE,PREVIEW_SIZE)))

In [ ]:
# Pivot extraction from features space
pretrained_tree = create_PP_index(N_PIVOTS, PREFIX_LEN, "pretrained_features", "pretrained_pivots_compare", "pretrained_permutations_compare", "pretrained_reordered_features_compare", "labels", "pretrained_reordered_labels_compare", k_medoids = True, chunksize = 1000, on_disk = False, num_indexes = 1)

In [ ]:
PREFIX_TREES = pretrained_tree
PIVOTS = "pretrained_pivots_compare"
REORDERED_FEATURES = "pretrained_reordered_features_compare"
REORDERED_LABELS = "pretrained_reordered_labels_compare"

evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

In [ ]:
# Pivot extraction from features space
finetuned_tree = create_PP_index(N_PIVOTS, PREFIX_LEN, "finetuned_features", "finetuned_pivots_compare", "finetuned_permutations_compare", "finetuned_reordered_features_compare", "labels", "finetuned_reordered_labels_compare", k_medoids = True, chunksize = 1000, on_disk = False, num_indexes = 1)

In [ ]:
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "finetuned_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

PREFIX_TREES = finetuned_tree
PIVOTS = "finetuned_pivots_compare"
REORDERED_FEATURES = "finetuned_reordered_features_compare"
REORDERED_LABELS = "finetuned_reordered_labels_compare"

evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

#STEP 10: FLASK WEB USER INTERFACE
In order to create the Web App, *Flask* is employed. *Ngrok* is used to expose the server running on the virtual machine to a public URL, as described in [ngrok tutorial](https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b#:~:text=Here%20comes%20the%20Python%20library%20flask%2Dngrok.&text=A%20secure%20URL%20is%20provided,before%20deploying%20it%20into%20production.).

In [ ]:
"""
MODEL LOADING AND VARIABLES DEFINITION
Load the model that will be used to extract the features from the query image.
"""

model = load_model(os.path.join(MODELS_PATH, "facenet_ft2.h5"))

K = 100
N_CANDIDATES = 300
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
PREFIX_LEN = 4
CHUNKSIZE = 1000

prefix_tree_web = PrefixTree("Prefix Tree WEB")
PrefixTree.add_from_csv(prefix_tree_web, "finetuned_permutations_0.csv", PREFIX_LEN, chunksize = CHUNKSIZE)
PrefixTree.merge_leaves(prefix_tree_web)
PREFIX_TREES = [prefix_tree_web]
REORDERED_FEATURES = "finetuned_reordered_features"
REORDERED_LABELS = "finetuned_reordered_labels"

In [ ]:
"""
PREPROCESSING FUNCTION DEFINITION
This function will be used to perform the preprocessing on the query image. 
The type of preprocessing used is exactly the same used in feature extraction phase.
"""

def preprocess(images, labels):
  # rescales from [0, 255] to [-1, 1], equivalent to:  images = (images / 127.5) - 1
  images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
  return images, labels

In [ ]:
"""
WEB APP DEFINITION AND DEPLOYING
In order to use the Web App, click on the second link.
"""

app = Flask(__name__,template_folder='/content/gdrive/MyDrive/templates',static_url_path='/content/gdrive/MyDrive/static')
run_with_ngrok(app)
app.config['UPLOAD_EXTENSIONS'] = ['.jpg', '.png', '.gif']
results=[]

@app.route('/')
def index():
    print(app.instance_path)
    return render_template('index.html',results=results)

@app.route('/search',methods=['POST'])
def web_search():
    uploaded_file = request.files['query']
    filename = uploaded_file.filename

    #remove all files from upload directory
    files = glob.glob('/content/gdrive/MyDrive/static/upload/*')
    for f in files:
      os.remove(f)
    
    if filename != '':

        #the uploaded file is accepted only if its extension is .jpg, .png or .gif
        file_ext = os.path.splitext(filename)[1]
        if file_ext not in app.config['UPLOAD_EXTENSIONS']:
            abort(400)
        
        #save query file in upload directory, necessary to correctly reconstruct it into a tensor
        file_path = os.path.join('/content/gdrive/MyDrive/static/upload', filename)
        uploaded_file.save(file_path)

        #read query image using image_dataset_from_directory, in order to perform the same preprocessing applied for feature extraction
        queryds, query_path = image_dataset_from_directory(directory='/content/gdrive/MyDrive/static/', labels="inferred", label_mode="int", color_mode="rgb", batch_size=BATCH_SIZE, image_size=(160,160), shuffle=False, seed = SEED)
        queryds = queryds.map(preprocess, deterministic=True)

        #extract features from query image using the specified model
        query_features = extract_features(model, queryds)

        #search for results 
        paths, scores, labels = search(query_features, K, pivot_file="finetuned_pivots", reordered_features_file=REORDERED_FEATURES, reordered_labels_file=REORDERED_LABELS, attachID = True)

        #transform the format so that it is understandable from html file
        results = [ [x, np.round(y,3), CATEGORICAL_LABELS[z]] for x,y,z in zip(paths[0], scores[0], labels[0])]
  
    return render_template('index.html',results=results)

if __name__== "__main__":
  app.run()

#STEP 11: OPTIMIZATIONS

## K-Medoids VS Random Pivots Selection

In [ ]:
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"

In [ ]:
N_PIVOTS = 7
PREFIX_LEN = 4
PAM_root = create_PP_index(N_PIVOTS, PREFIX_LEN, "finetuned_features", "finetuned_pivots_medoids", "finetuned_permutation_medoids", "finetuned_reordered_features_medoids", "labels", "finetuned_reordered_labels_medoids", k_medoids = True, chunksize = 1000, on_disk = False, num_indexes = 1)

In [ ]:
N_PIVOTS = 10
PREFIX_LEN = 4
random_root = create_PP_index(N_PIVOTS, PREFIX_LEN,'finetuned_features', 'finetuned_pivots_random', "finetuned_permutations_random", 'reordered_finetuned_features_random','labels', 'reordered_finetuned_labels_random', chunksize = 1000, on_disk = False, num_indexes = 1)

In [ ]:
N_QUERIES = 7
K = 100
N_CANDIDATES = 300
PREVIEW_SIZE = 100

test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

# One for each class
query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]

# Random
# query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random

queries_labels = test_Labels[query_indexes]
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "finetuned_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

for fetures, label in zip(queries_features, queries_labels):
  print(CATEGORICAL_LABELS[label])
  display(load_img(fetures[0], target_size=(PREVIEW_SIZE,PREVIEW_SIZE)))

In [ ]:
PREFIX_TREES = PAM_root
PIVOTS = "finetuned_pivots_medoids"
REORDERED_FEATURES = "finetuned_reordered_features_medoids"
REORDERED_LABELS = "finetuned_reordered_labels_medoids"

evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

In [ ]:
PREFIX_TREES = random_root
PIVOTS = "finetuned_pivots_random"
REORDERED_FEATURES = "reordered_finetuned_features_random"
REORDERED_LABELS = "reordered_finetuned_labels_random"

evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

##Perturbations VS Non Perturbations

In [ ]:
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
N_PIVOTS = 7
PREFIX_LEN = 4

In [ ]:
N_PIVOTS = 7
PREFIX_LEN = 4
pert_root = create_PP_index(N_PIVOTS, PREFIX_LEN, "finetuned_features", "finetuned_pivots_pert", "finetuned_permutation_pert", "finetuned_reordered_features_pert", "labels", "finetuned_reordered_labels_pert", k_medoids = True, chunksize = 1000, on_disk = False, num_indexes = 1)

In [ ]:
N_QUERIES = 7
K = 100
N_CANDIDATES = 300
PREVIEW_SIZE = 100
PREFIX_TREES = pert_root
PIVOTS = "finetuned_pivots_pert"
REORDERED_FEATURES = "finetuned_reordered_features_pert"
REORDERED_LABELS = "finetuned_reordered_labels_pert"

test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

# One for each class
query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]

# Random
# query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random

queries_labels = test_Labels[query_indexes]
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "finetuned_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

for fetures, label in zip(queries_features, queries_labels):
  print(CATEGORICAL_LABELS[label])
  display(load_img(fetures[0], target_size=(PREVIEW_SIZE,PREVIEW_SIZE)))

In [ ]:
evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

In [ ]:
evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=True)

##Multiple Indexes

In [ ]:
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"

In [ ]:
N_PIVOTS = 10
PREFIX_LEN = 4
random_roots = create_PP_index(N_PIVOTS, PREFIX_LEN,'finetuned_features', 'finetuned_pivots_multiple_idexes', "finetuned_permutations_multiple_idexes", 'reordered_finetuned_features_multiple_idexes','labels', 'reordered_finetuned_labels_multiple_idexes', k_medoids = False, num_indexes=3)

In [ ]:
N_PIVOTS = 7
PREFIX_LEN = 4
pam_roots = create_PP_index(N_PIVOTS, PREFIX_LEN,'finetuned_features', 'finetuned_pivots_multiple_idexes_medoids', "finetuned_permutations_multiple_idexes_medoids", 'reordered_finetuned_features_multiple_idexes_medoids','labels', 'reordered_finetuned_labels_multiple_idexes_medoids', k_medoids = True, num_indexes=3)

In [ ]:
N_QUERIES = 7
K = 100
N_CANDIDATES = 300
PREVIEW_SIZE = 100

test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

# One for each class
query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]

# Random
# query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random

queries_labels = test_Labels[query_indexes]
test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "finetuned_features_private.csv"), sep=',',header=None))
queries_features = test_features[query_indexes]

for fetures, label in zip(queries_features, queries_labels):
  print(CATEGORICAL_LABELS[label])
  display(load_img(fetures[0], target_size=(PREVIEW_SIZE,PREVIEW_SIZE)))

In [ ]:
PREFIX_TREES = random_roots
PIVOTS = "finetuned_pivots_multiple_idexes"
REORDERED_FEATURES = "reordered_finetuned_features_multiple_idexes"
REORDERED_LABELS = "reordered_finetuned_labels_multiple_idexes"

evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

In [ ]:
PREFIX_TREES = pam_roots
PIVOTS = "finetuned_pivots_multiple_idexes_medoids"
REORDERED_FEATURES = "reordered_finetuned_features_multiple_idexes_medoids"
REORDERED_LABELS = "reordered_finetuned_labels_multiple_idexes_medoids"

evaluate_knn_search(queries_features, queries_labels, K, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, from_zip=True, query_perturbation=False)

##Search for Best Combination of Pivot Number and Prefix Lenght





In [ ]:
def search_candidates(query_features, prefix_tree, pivots, Z, features_file, labels_file,PREFIX_LEN):

  query_prefix = compute_permutation_prefix(query_features, PREFIX_LEN, pivots)
  subtree = PrefixTree.find_prefix(prefix_tree, query_prefix, Z)
  PrefixTree.print_tree(subtree)
  leaves = PrefixTree.get_leaves(subtree)
  df_features = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,features_file), sep=',', header=None, skiprows= leaves[0], nrows=(leaves[-1] - leaves[0] +1)).to_numpy()
  df_labels = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,labels_file), sep=',', header=None, skiprows= leaves[0], nrows=(leaves[-1] - leaves[0] +1)).to_numpy().reshape(-1)

  return df_features, df_labels

In [ ]:
def search(queries_features, K, PREFIX_TREE, N_CANDIDATES, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, PREVIEW_SIZE, PREFIX_LEN, from_zip = True, attachID = False):
  '''
  Search for k best images similar to queries using PP-Index
  Returns topk_images_list, topk_scores_list, topk_labels_list
  '''

  topk_images_list = []
  topk_scores_list = []
  topk_labels_list = []
  n_candidates = []

  for query in tqdm(queries_features):
    candidates, candidates_labels = search_candidates(query, PREFIX_TREE, PIVOTS, N_CANDIDATES, REORDERED_FEATURES, REORDERED_LABELS,PREFIX_LEN)
    topk_paths,  topk_scores, topk_labels= k_nn_search(query, candidates, candidates_labels, K, from_zip, attachID)
    n_candidates.append(len(candidates))

    if not attachID:
      topk_images = [img_to_array(load_img(path, target_size=(PREVIEW_SIZE,PREVIEW_SIZE))) for path in  topk_paths]
      topk_images_list.append(topk_images)
    else:
      topk_images_list.append(topk_paths)
      
    topk_scores_list.append(topk_scores)
    topk_labels_list.append(topk_labels)

  return topk_images_list, topk_scores_list, topk_labels_list, n_candidates

In [ ]:
def evaluate_knn_search_return_ap(queries_features, queries_labels, k, PREFIX_TREE, N_CANDIDATES, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, PREVIEW_SIZE,PREFIX_LEN, from_zip=True):
  '''
  Evaluate APs and mean AP of all queries
  '''

  # Search 
  topk_images, topk_scores, topk_labels,n_candidates = search(queries_features, k, PREFIX_TREE, N_CANDIDATES, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, PREVIEW_SIZE,PREFIX_LEN, from_zip)
  # If a certain retreived image is relevant 
  topk_is_relevant = np.array(topk_labels) == queries_labels.reshape(-1,1)

  # Average Precision Score computation
  aps = np.array([sklearn.metrics.average_precision_score(l, s) for l,s in zip(topk_is_relevant, topk_scores)])
  aps[np.isnan(aps)] = 0
  print('APs per Query:', aps)
  print('mAP:', np.mean(aps))

  return np.mean(aps), n_candidates

In [ ]:
#map, recall, number of candidates

BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
CHUNKSIZE = 1000

def get_map_candidates(pivots):

  global_map = []
  global_n_candidates = []

  for pivot in pivots:

    pivot_map = []
    pivot_candidates = []

    PREFIX_LEN = list(range(2,pivot+1))

    extract_random_pivots('finetuned_features.csv','optimized_pivots.csv', pivot)
    #extract_k_medoids_pivots('finetuned_features.csv', 'optimized_pivots.csv', 25000, pivot)

    # Dataset of images permutations of indexes 
    create_permutation_dataset("finetuned_features.csv", "optimized_permutations.csv", "optimized_pivots.csv", CHUNKSIZE)

    for prefix in PREFIX_LEN:

      # Creates Empty Prefix Tree
      prefix_tree_root = PrefixTree("Prefix Tree Optimized"+str(pivot)+"-"+str(prefix))
      # Adds all images pivots permutations in "permutations.csv"
      PrefixTree.add_from_csv(prefix_tree_root, "optimized_permutations.csv", prefix, chunksize = 1000)

      # Get images indexes ordered by the prefix tree
      indexes_reordered_by_prefix_tree = PrefixTree.get_leaves(prefix_tree_root)
      print(indexes_reordered_by_prefix_tree)

      # Reorder indexes to obtain a mapping necessary to quick memory sort
      indexing_mapping = [indexes_reordered_by_prefix_tree.index(i) for i in tqdm(range(len(indexes_reordered_by_prefix_tree)))]
      np.save(os.path.join(BASE_PP_INDEX_PATH,"indexing_mapping_opt"), indexing_mapping)
      print(indexing_mapping)

      # In memory features and labels reordering
      reorder_datastore_in_memory('finetuned_features.csv', 'reordered_optimized_finetuned_features.csv', indexing_mapping)
      reorder_datastore_in_memory('labels.csv', 'reordered_optimized_finetuned_labels.csv', indexing_mapping)

      # Merge leaves in order to have only start and end index for a certain permutation
      PrefixTree.merge_leaves(prefix_tree_root)
      PrefixTree.print_tree(prefix_tree_root)

      N_QUERIES = 10
      K = 100
      N_CANDIDATES = 300
      PREFIX_TREE = prefix_tree_root
      PIVOTS = np.array(pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,"optimized_pivots.csv"), sep=',',header=None))
      REORDERED_FEATURES = "reordered_optimized_finetuned_features.csv"
      REORDERED_LABELS = "reordered_optimized_finetuned_labels.csv"
      PREVIEW_SIZE = 100

      #sarà private test dataset
      test_features = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "finetuned_features_private.csv"), sep=',',header=None))
      test_Labels = np.array(pd.read_csv(os.path.join(FEATURES_PATH, "private_test_labels.csv"), sep=',',header=None)).reshape(-1)

      # query_indexes = [10, 11]
      # query_indexes = [14, 78, 79, 12, 18]
      query_indexes = [100, 500,  700, 1200,  2200, 2900, 3400]
      #query_indexes = np.random.choice(len(test_features), N_QUERIES, replace=False)  # get n queries at random
      queries_features = test_features[query_indexes]
      queries_labels = test_Labels[query_indexes]

      map, n_candidates = evaluate_knn_search_return_ap(queries_features, queries_labels, K, PREFIX_TREE, N_CANDIDATES, PIVOTS, REORDERED_FEATURES, REORDERED_LABELS, PREVIEW_SIZE,prefix)
      pivot_map.append(map)
      pivot_candidates.append(n_candidates)

    global_map.append(pivot_map)
    global_n_candidates.append(pivot_candidates)


  return global_map, global_n_candidates

In [ ]:
map, candidates = get_map_candidates([5,6,7,8,9,10])

###Random Pivot Selection Experiment

In [ ]:
for elem in map:
  print(elem)


In [ ]:
for elem in candidates:
  elem =np.array(elem)
  print(np.mean(elem,axis=1).round())

###K-Medoids Pivot Selection Experiment

In [ ]:
for elem in map:
  print(elem)


In [ ]:
for elem in candidates:
  elem =np.array(elem)
  print(np.mean(elem,axis=1).round())

###Results Evaluation
In order to compare the results, a summary score is derived from MAP and MCANDIDATES. This score is used to make a quick comparision, and it has not definitive importance. In particular:

\begin{equation}
score= weight_{map}*MAP + weight_{mcandidates}*( 1 - normalized\_MCANDIDATES )
\end{equation}

where: <br>
$MAP$: Mean Average Precision for a given configuration <br>
$MCANDIDATES$: Mean Number of Candidates considered <br>
$normalized\_MCANDIDATES$: min-max normalization of MCANDIDATES assuming 300 as minimum value (300 is the minimum number of candidates) and 57298 as as maximum value (cardinality of the whole dataset, private test set excluded). <br>
$weight_{map}$: fixed to $\frac{2}{3}$ <br>
$weight_{mcandidates}$: fixed $\frac{1}{3}$ <br>

In [ ]:
"""
CREATION OF THE ARRAY CONTAINING PIVOT-PREFIX PAIRS
Number of pivots varies between 5 and 10, while prefix varies from 2
to the number of pivots considered. 
"""

pivots = np.arange(5,11)
pivot_prefix_array = []

for pivot in pivots:

  pivot_array = []

  for prefix in np.arange(2,pivot+1):

    pivot_array.append([pivot, prefix])

  pivot_prefix_array.append(pivot_array)

In [ ]:
"""
SCORES COMPUTATION FOR K-MEDOIDS-PIVOT-SELECTION EXPERIMENT
Scores for K-Medoids experiments are computed according to the formula described above.
"""


maps2 = np.array([ [0.42797970216219827, 0.4473705587136627, 0.4405917904811597, 0.4405917904811597],
                   [0.39857218910802683, 0.3938975786910332, 0.3893925133982128, 0.38617527397027535, 0.38617527397027535],
                   [0.4222522247273362, 0.42691657055673315, 0.45032691652410195, 0.41592846774276016, 0.40809568227905463, 0.40809568227905463],
                   [0.3435707089773567, 0.3616436592036507, 0.3637343130382096, 0.36100876507821483, 0.35546469664712027, 0.35546469664712027, 0.35546469664712027],
                   [0.35632062471054693, 0.36425998668564935, 0.35709273083158255, 0.36129285725856247, 0.36636986992321263, 0.3650510393241892, 0.3649330819984303, 0.3649330819984303],
                   [0.4223070580277098, 0.4204660304954248, 0.41989962124115127, 0.4209338620053527, 0.4209338620053527, 0.4398023948565452, 0.4398023948565452, 0.4398023948565452, 0.4398023948565452] ], dtype=object)

mcandidates2 = np.array([ [5087, 3454, 2371, 2371],
                          [5681, 4923, 4399, 3736, 3736],
                          [2864, 2112, 1568, 1309, 1175, 1175],
                          [2998, 2264, 2098, 2073, 1974, 1970, 1970],
                          [3595, 2942, 2738, 2495, 2457, 2417, 2414, 2414],
                          [3821, 2827, 2528, 2482, 2476, 2473, 2473, 2473, 2473] ],dtype=object)

#300 is the minimum number of candidates considered, while 57298 is the cardinality of the whole dataset
min=300
max=57298

map_weight = 2/3
mcandidate_weight = 1/3

mcandidates2_normalized = copy.deepcopy(mcandidates2)

#min-max normalization of MCANDIDATES using 300 as min value and 57298 as max value
for i in np.arange(pivots.size):

  for j in np.arange(len(mcandidates2[i])):

    mcandidates2_normalized[i][j] = (mcandidates2_normalized[i][j]-min)/(max-min)

#scores computation according to the formula described above
scores2 = []

for i in np.arange(pivots.size):

  pivot_scores = []

  for j in np.arange(len(mcandidates2_normalized[i])):

    map = maps2[i][j]
    mcandidate_normalized = mcandidates2_normalized[i][j]
    score = (map_weight*map+mcandidate_weight*(1-mcandidate_normalized))
    pivot_scores.append(score)

  scores2.append(pivot_scores)

#print the scores
for i in np.arange(pivots.size):

  for j in np.arange(len(scores2[i])):

    print("PIVOTS: " + str(pivot_prefix_array[i][j][0]) + " PREFIX: "+ str(pivot_prefix_array[i][j][1]) + "\t SCORE: " + str(np.round(scores2[i][j],5)) + 
          "\t CANDIDATES: " + str(mcandidates2[i][j]) + "\t NORMALIZED CANDIDATES: " + str(np.round(mcandidates2_normalized[i][j],5)) + "\t MAP: " + str(np.round(maps2[i][j],5)))

In [ ]:
"""
SCORES COMPUTATION FOR RANDOM-PIVOT-SELECTION EXPERIMENT
Scores for Random pivots extraction experiments are computed according to the formula described above.
"""

maps3 = np.array( [ [0.5159362375487182, 0.4209606316965547, 0.3879634867332293, 0.3879634867332293],
                    [0.46054902825532285, 0.36566337362714, 0.37835149760185427, 0.3502531769493248, 0.3502531769493248],
                    [0.5002595477724913, 0.4640710175155756, 0.46451728465019315, 0.4662482028715962, 0.46547039141841834, 0.46547039141841834],
                    [0.42187718416723957, 0.43694466854882236, 0.4228089425608541, 0.4257841391972833, 0.4257796505718903, 0.4257796505718903, 0.4257796505718903],
                    [0.48502996209122184, 0.4872694411078049, 0.4780190972542616, 0.46919987803258933, 0.4645432222904482, 0.4645432222904482, 0.46515521596576204, 0.46515521596576204],
                    [0.44521338037565844, 0.4681250592452478, 0.5074289379589547, 0.5074289379589547, 0.5074289379589547, 0.5074289379589547, 0.5074289379589547, 0.5074289379589547, 0.5074289379589547] ], dtype=object)

mcandidates3= np.array([ [6033, 4256, 3766, 3766],
                   [6146, 1793, 1010,  626,  626],
                   [4270, 2194, 1980, 1711, 1698, 1698],
                   [1874, 1137,  775,  595,  595,  595,  595],
                   [4724, 2135, 1637, 1536, 1480, 1477, 1475, 1475],
                   [2550, 1874, 1663, 1663, 1663, 1663, 1663, 1663, 1663] ], dtype=object)

#300 is the minimum number of candidates considered, while 57298 is the cardinality of the whole dataset
max = 57298
min = 300

map_weight = 2/3
mcandidate_weight = 1/3

mcandidates3_normalized = copy.deepcopy(mcandidates3)

#min-max normalization of MCANDIDATES using 300 as min value and 57298 as max value
for i in np.arange(pivots.size):

  for j in np.arange(len(mcandidates3[i])):

    mcandidates3_normalized[i][j] = (mcandidates3_normalized[i][j]-min)/(max-min)

scores3 = []

#scores computation according to the formula described above
for i in np.arange(pivots.size):

  pivot_scores = []

  for j in np.arange(len(mcandidates3_normalized[i])):

    map = maps3[i][j]
    mcandidates_normalized = mcandidates3_normalized[i][j]
    score = (map_weight*map+mcandidate_weight*(1-mcandidates_normalized))
    pivot_scores.append(score)

  scores3.append(pivot_scores)

#print the scores
for i in np.arange(pivots.size):

  for j in np.arange(len(scores3[i])):

    print("PIVOTS: " + str(pivot_prefix_array[i][j][0]) + " PREFIX: "+ str(pivot_prefix_array[i][j][1]) + "\t SCORE: " + str(np.round(scores3[i][j],5)) + 
          "\t CANDIDATES: " + str(mcandidates3[i][j]) + "\t NORMALIZED CANDIDATES: " + str(np.round(mcandidates3_normalized[i][j],5)) + "\t MAP: " + str(np.round(maps3[i][j],5)))